<h1>Project #7: Counting Syllables</h1>
<p>Python code to count the number of syllables in an English word</p>
<a href="https://github.com/cmusphinx/cmudict"><p>Carnegie Melon Pronouncing Dictionary</p></a>

- take user input
- open syllable count doc
- search through dict for each word
- count phonetic syllables associated with each word
- return count

The following code tests a function to count the number of syllables using NLTK and cmudict. This function will be used in a later project.

In [2]:
import sys
from string import punctuation
import json
import nltk
nltk.download("cmudict")
from nltk.corpus import cmudict

cmudict = cmudict.dict()

with open("../resources/missing_words.json") as f:
    missing_words = json.load(f)

def count_syllables(words):
    num_sylls = 0
    for word in words.lower().split():
        if word in missing_words:
            num_sylls += missing_words[word]
        elif word in cmudict.keys():
            #aachener: [[AA1 K AH0 N ER0]]
            for phonemes in cmudict[word][0]:
                for phoneme in phonemes:
                    if phoneme.isnumeric():
                        num_sylls +=1
        else:
            num_sylls += int(input("Sorry! We don't have that word. Enter the number of syllables:"))
    return num_sylls



word = input("Enter a word to count syllables")
num_syllables = count_syllables(word)
print(num_syllables)

[nltk_data] Downloading package cmudict to /Users/eli/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


2


<h1>Project #8: Writing Haikus</h1>
<p>Use Markov chain analysis to predict the next word in a series based on the current words.</p>
<p>This project creates a Markov model based on user input.</p>
<p>Markov model order 1: uses single word to find next word.</p>
<p>Markov model order 2: uses 2 words to find next word.</p>
<p>We have to use markov model order 1 first to then be able to use order 2 with 2 words.</p>
<p>Start a sentence with a random word, check syllables, and predict a word using Markov</p>
<p>disregard Order 2 Markov Model. Clean up corpus (punctuation). Write a function that generates a line of poetry with Model. Takes either 5 or 7 syllables as an input. Maybe while loop</p>

- Markov model is built by grabbing a word and making a list with the words that follow it. 2nd order is built in the same manner but with a 2 word phrase instead of a single one

In [3]:
import sys
import logging
import random
from collections import defaultdict

with open("../resources/train.txt",'r') as f:
    raw_haiku = f.read()

corpus = raw_haiku.replace("\n",' ')
corpus = corpus.replace("!","")
corpus = corpus.replace(",","")
corpus = corpus.replace("?","")
corpus = corpus.split()

def map_word_to_word(corpus):
    limit = len(corpus) - 1
    dict1_to_1 = defaultdict(list)
    for index, word in enumerate(corpus):
        if index < limit:
            suffix = corpus[index+1]
            dict1_to_1[word].append(suffix)
    return dict1_to_1

def map2words_to_word(corpus):
    limit = len(corpus) - 2
    dict2_to_1 = defaultdict(list)
    for index, word in enumerate(corpus):
        if index < limit:
            key = word + " " + corpus[index+1]
            suffix = corpus[index+2]
            dict2_to_1[key].append(suffix)
    return dict2_to_1

def randomWord(corpus):
    word = random.choice(corpus)
    num_sylls = count_syllables(word)
    return word, num_sylls

firstOrderMarkovModel = map_word_to_word(corpus)

secondOrderMarkovModel = map2words_to_word(corpus)

randomWord(corpus) #returns word with number of syllables

('temples', 2)

<p>Now that we have a Markov model to predict the next word in a sentence, we can begin to put that to use and write some haiku lines. Firstly, we need a function to generate the next word based on the previous word. (The second order Markov Model will have to be implemented later, and with a seperate function, the logic is kicking my ass at the moment for a first order Markov.) Generating a word is only the first step, generating multiple, within the syllable restrictions, is the second step.</p>
<ul>
<li>Syllables</li>
<li>Sanity</li>
<li>Grammar</li>
</ul>

In [17]:
def word_after(word, currentSylls, targetSylls,suffixMap=firstOrderMarkovModel):
    acceptedWords = []
    suffixes = suffixMap[word]
    for candidate in suffixes:
        num_sylls = count_syllables(candidate)
        if currentSylls + num_sylls <= targetSylls:
            acceptedWords.append(candidate)
    return acceptedWords
def word_after2(words, currentSylls, targetSylls,suffixMap=secondOrderMarkovModel):
    acceptedWords = []
    suffixes = suffixMap[words]
    for candidate in suffixes:
        numSyllables = count_syllables(candidate)
        if currentSylls + numSyllables <= targetSylls:
            acceptedWords.append(candidate)
    return acceptedWords

word, num_sylls1 = randomWord(corpus)

word_after(word, num_sylls1,5)

word_after2("these dark", 2,5)

['waters',
 'hands',
 'hands',
 'waters',
 'hands',
 'waters',
 'hands',
 'hands',
 'waters',
 'hands',
 'hands',
 'waters',
 'waters',
 'waters',
 'waters',
 'hands',
 'hands',
 'hands',
 'waters',
 'waters',
 'waters',
 'waters',
 'waters',
 'hands',
 'hands',
 'hands',
 'hands',
 'waters',
 'waters',
 'hands',
 'hands',
 'waters',
 'waters',
 'hands',
 'hands']

<p>Now that we have a function to predict the next word in a haiku, we need a function to string multiple words together and count the syllables</p>

In [31]:
def writeLine(seedWord, targetSyllables,suffixMap):
    line = seedWord
    currentSyllables = count_syllables(seedWord)
    while currentSyllables <= targetSyllables:
        if len(line.split()) < 2:
            words = word_after(seedWord, currentSyllables, targetSyllables)
            print(line)
        elif len(line.split()) >1:
            line = line.split()
            words = word_after2(line[-1] + " " + line[-2],currentSyllables,targetSyllables)
            print(line)
        if len(words) > 0:
            choice = random.choice(words)
        print(line)
        for word in line:
            line += word
        line += " " + choice
        currentSyllables = count_syllables(line)
    return line
writeLine('dark',7,firstOrderMarkovModel)

dark
dark


ValueError: invalid literal for int() with base 10: ''

<p>So now we can call writeLine and add the results together to form a haiku</p>

In [21]:
writeLine("summer",5,firstOrderMarkovModel) + "/" + writeLine("dark", 7,firstOrderMarkovModel) + "/" + writeLine("yellow",5,firstOrderMarkovModel)

'summer shower night night/dark clouds arise hut fields clouds clouds/yellow bamboo bamboo'